In [2]:
import json
import pandas as pd
import numpy as np
from database.operations.chess_com_api import get_profile
import requests
from dotenv import load_dotenv
import os
import psycopg2
from bs4 import BeautifulSoup
import joblib
from database.operations.chess_com_api import download_months, download_month, month_of_games
#from database.operations.players import read_oponents, read_all_players
from database.database.db_interface import DBInterface
from database.database.models import Player, Game
from database.database.engine import DBSession
from database.operations.models import PlayerCreateData, GameCreateData
from database.operations.format_dates import create_range
load_dotenv(".env")
import io
import concurrent

In [3]:
get_profile('AlexandraBotez')

{'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/28583276.b0fa66c3.200x200o.e193184a1fb4.jpg',
 'player_id': 28583276,
 'url': 'https://www.chess.com/member/AlexandraBotez',
 'name': 'Alexandra Botez',
 'title': 'WFM',
 'followers': 66297,
 'location': 'New York',
 'joined': 1465620232,
 'status': 'premium',
 'is_streamer': True,
 'twitch_url': 'https://twitch.tv/botezlive',
 'verified': False,
 'league': 'Champion',
 'country': 'US'}

In [2]:
os.listdir(os.getcwd())

['.env',
 '.ipynb_checkpoints',
 'database',
 'games.ipynb',
 'games.pkl',
 'main.py',
 'player.ipynb',
 '__pycache__']

In [3]:
a = joblib.load('games.pkl')

In [4]:
a[0]

'[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2021.02.01"]\n[Round "-"]\n[White "Pat_Buchanan"]\n[Black "Ichbinalex"]\n[Result "1-0"]\n[CurrentPosition "r2q1rk1/pbpn1ppp/1p1ppn2/8/1P1PP3/2NB1N2/1PP2PPP/R1BQR1K1 b - -"]\n[Timezone "UTC"]\n[ECO "B00"]\n[ECOUrl "https://www.chess.com/openings/Owens-Defense"]\n[UTCDate "2021.02.01"]\n[UTCTime "00:45:34"]\n[WhiteElo "1379"]\n[BlackElo "1404"]\n[TimeControl "60"]\n[Termination "Pat_Buchanan won by resignation"]\n[StartTime "00:45:34"]\n[EndDate "2021.02.01"]\n[EndTime "00:45:51"]\n[Link "https://www.chess.com/game/live/6410917729"]\n\n1. e4 {[%clk 0:00:59.9]} 1... b6 {[%clk 0:00:59.9]} 2. Nf3 {[%clk 0:00:59]} 2... Bb7 {[%clk 0:00:59.6]} 3. Nc3 {[%clk 0:00:58.5]} 3... e6 {[%clk 0:00:59]} 4. d4 {[%clk 0:00:58.2]} 4... Bb4 {[%clk 0:00:58.6]} 5. Bd3 {[%clk 0:00:57.6]} 5... Nf6 {[%clk 0:00:57.5]} 6. O-O {[%clk 0:00:56.2]} 6... O-O {[%clk 0:00:56.7]} 7. Re1 {[%clk 0:00:55.6]} 7... d6 {[%clk 0:00:55.2]} 8. a3 {[%clk 0:00:54.5]} 8... Nbd7 {[%clk

In [5]:
s = {'id': 6136244019,
     'start_time': '23:35:27',
     'year': 2021,
     'month': 1,
     'day': 3,
     'white': 'pat_buchanan',
     'black': 'abelcenteno99',
     'white_elo': '1166',
     'black_elo': '1079',
     'white_result': 1,
     'black_result': 0,
     'time_control': '60',
     'eco': 'b06',
     'time_elapsed': '00:01:51',
     'termination': 'time',
     'n_moves': 29}

In [9]:
w = GameCreateData(**s)

In [13]:
w.model_dump()

{'id': 6136244019,
 'start_time': '23:35:27',
 'year': 2021,
 'month': 1,
 'day': 3,
 'white': 'pat_buchanan',
 'black': 'abelcenteno99',
 'white_elo': 1166,
 'black_elo': 1079,
 'white_result': 1.0,
 'black_result': 0.0,
 'time_control': '60',
 'eco': 'b06',
 'time_elapsed': '00:01:51',
 'termination': 'time',
 'n_moves': 29}

In [ ]:
USER_AGENT = os.getenv('USER_AGENT')
DOWNLOAD_MONTH = (
    os.getenv("DOWNLOAD_MONTH")
    .replace("{player_name}", 'hikaru')
    .replace("{year}", str(2021))
    .replace("{month}", str(11))
)
games = requests.get(DOWNLOAD_MONTH,
                     allow_redirects=True,
                     timeout=3,
                     headers = {"User-Agent": USER_AGENT})

In [ ]:
def some(params):
    params['pipi'].append('opipipi')

In [ ]:
result = [] 
params = [{'pipi':result} for i in range(2)]

In [ ]:
params

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        executor.map(some, params)

In [ ]:
result

In [ ]:
def download_months(player_name, valid_dates):
    """
    Ask chessdotcom for the games on the date_range trough a threadpool
    returns valid games for valid months and a list of months asked
    """
    return_games = []
    params = [
        {
            "player_name": player_name,
            "year": date[0],
            "month": date[1],
            "return_games": return_games,
        }
        for date in valid_dates
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(month_of_games, params)
    print(f"GOT {len(return_games)} games")
    print("downloading over")
    #return_games = [game for game in return_games if game is not False]
    print(f'returned months = {len(return_games)}')
    return return_games

In [30]:
a = download_months('pat_buchanan', [(2021,2)])

2021 2
GOT 1104 games
downloading over
returned games = 1104


In [ ]:
a

In [ ]:
pgn = io.StringIO(games.content.decode().replace("'", '"'))

In [ ]:
data = {"player_name":"caruana", "dates":"2021_1-2021_2"}

In [ ]:
date_range = create_range(data)

In [ ]:
date_range

In [ ]:
pgn = download_month(data["player_name"], 2021, 5)

In [ ]:
games = download_months('pat_buchanan',[(2021,1),(2021,2)])

In [ ]:
games

In [ ]:
profile= {'player_name':'pipi'}
profile = PlayerCreateData(**profile)

In [ ]:
profile

In [ ]:
d = joblib.load('games.pkl')

In [ ]:
len(d)

In [ ]:
type(d[0])

In [ ]:
[i for i in d if type(d)==tuple]

In [ ]:
load_dotenv()
CONN_STRING = os.getenv("CONN_STRING")

In [ ]:
from database.database.engine import init_db

In [ ]:
init_db(CONN_STRING)

In [ ]:
player_interface = DBInterface(Player)
game_interface = DBInterface(Game)

In [ ]:
player_interface.player_exists('mishanick')

In [ ]:
player_interface.read_all()

In [ ]:
session = DBSession()
white = session.query(
    game_interface.db_class
).where(game_interface.db_class.white =='pat_buchanan').all()
black = session.query(
    game_interface.db_class
).where(game_interface.db_class.black =='pat_buchanan').all()

In [ ]:
white[0].black

In [ ]:
black[0].white

In [ ]:
white + black

In [ ]:
def get_pgn_item(game, item: str) -> str:
    if item == "Termination":
        return (
            game.split(f"{item}")[1]
            .split("\n")[0]
            .replace('"', "")
            .replace("]", "")
            .lower()
        )
    return (
        game.split(f"{item}")[1]
        .split("\n")[0]
        .replace('"', "")
        .replace("]", "")
        .replace(" ", "")
        .lower()
    )

In [ ]:
def clean_games(game: str) -> bool:
    """
    It validates games for length of moves and formatting errors

    """
    game_with_more_than_n_moves = "10. "  # format: 'INT. '

    try:
        game.split("\n\n")[1][:-4]
    except:
        return False
    if not game_with_more_than_n_moves in game.split("\n\n")[1][:-4]:
        return False
    if game.split("\n\n")[1][:-4].startswith("1... "):
        return False
    if (
        "/"
        in game.split(f"TimeControl")[1]
        .split("\n")[0]
        .replace('"', "")
        .replace("]", "")
        .replace(" ", "")
        .lower()
    ):
        return False
    return True

In [ ]:
valid_dates = [(2023, 1),(2023, 1),(2023, 1),(2023, 1),(2023, 1),(2023, 1)]

In [ ]:
def format_valid_dates(valid_dates):
    months = ''
    for date in valid_dates:
        format = f'{date[0]}_{date[1]}###'
        months += format
    return months

In [ ]:
format_valid_dates(valid_dates).split('###')

In [ ]:
player_name = 'hikaru'

In [ ]:
w = download_months(player_name, valid_dates)

In [ ]:
w[0]

In [ ]:
get_pgn_item(w[0], "Link")

In [ ]:
games = [game for game in w if clean_games(game)]

In [ ]:
len(w)

In [ ]:
int(get_pgn_item(w[0], "[Link").split("/")[-1])

In [ ]:
players_and_game_id = np.array(
        [
            [
                get_pgn_item(game, "White").lower(),
                get_pgn_item(game, "Black").lower(),
                int(get_pgn_item(game, "[Link").split("/")[-1]),
            ]
            for game in games
        ]
    )

In [ ]:
players_and_game_id

In [ ]:
def create_month(params: list) -> None:
    """
    Download a month, splits the games and return a list of pgn games.
    """
    pgn = download_month(params["player_name"], params["year"], params["month"])
    if pgn == False:
        params["return_games"].extend(False)
        return
    games = pgn.read().split("\n\n\n")
    return games

In [ ]:
pgn = download_month('hikaru', '2023', '1')

In [ ]:
pgn = pgn.read().split("\n\n\n")

In [2]:
def get_ask_connection():
    CONN_STRING = os.getenv("PSYCOPG2_CONN_STRING")
    return psycopg2.connect(CONN_STRING, port = 5433)


def player_exists_at_db(player_name: str):
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            f"select player_name from player where player_name='{player_name}'"
        )
        result = curs.fetchall()
    if len(result) == 1:
        return True
    return False
def get_games(player_name: str):
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            f"select id from game where white='{player_name}'"
        )
        result = curs.fetchall()
    return result
def get_all_players():
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            f"select player_name from player"
        )
        result = curs.fetchall()
    return result
def open_request(sql_question:str):
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            sql_question
        )
        result = curs.fetchall()
    return result

In [ ]:
open_request(f"select * from player")

In [26]:
mo = open_request(f"select dates from months where months.player_name = 'hikaru'")

In [27]:
len(mo)

0

In [28]:
if not mo:
    print('Mo')

Mo


In [11]:
dates

'2021_3###2021_2###2021_5###2021_4###2021_1###'

In [19]:
w = [6410917729, 6400362319, 6400292960, 6400092841, 6400010655, 6348792540, 6348712561, 6337108937, 6227713685, 6225888016, 6225817849, 6220966157, 6201443796, 6201013151, 6200999263, 6200973865, 6197653773, 6197453000, 6197411167, 6191966718, 6191875284, 6178893187, 6141476402, 6141219]


In [20]:
for i in w:
    print(open_request(f"select * from game where game.id = {i}"))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [26]:
a = [(2021,1),(2021,2), (2021,1)]
b = [(2021,1),(2021,6)]

In [27]:
set(a)

{(2021, 1), (2021, 2)}

In [28]:
c = a+b

In [29]:
set(c)

{(2021, 1), (2021, 2), (2021, 6)}

In [34]:
e = "2021_2###2021_3###2021_4###2021_5"

In [35]:
e = [(int(x.split('_')[0]),int(x.split('_')[1])) for x in e.split('###')]

In [36]:
e

[(2021, 2), (2021, 3), (2021, 4), (2021, 5)]

In [37]:
c

[(2021, 1), (2021, 2), (2021, 1), (2021, 1), (2021, 6)]

In [38]:
[x for x in c if x not in e]

[(2021, 1), (2021, 1), (2021, 1), (2021, 6)]

In [ ]:
get_all_players()

In [ ]:
player_exists_at_db('hikaru'.lower())

In [ ]:
get_profile('sdssssdfssssssssssssss')

In [ ]:
player_name = 'Pat_Buchanan'

In [ ]:
prof = get_profile(player_name)

In [ ]:
prof

In [ ]:
type(prof)

In [ ]:
country = prof.pop('country').split('/')[-1]

In [ ]:
country

In [ ]:
member_exists(player_name = 'hikaru')

In [ ]:
from database.operations.models import PlayerCreateData, PlayerResult

In [ ]:
profile = get_profile('pat_buchanan')

In [ ]:
profile.pop('@id')

In [ ]:
p_name = profile.pop('name')

In [ ]:
p_name

In [ ]:
profile

In [ ]:
from pydantic import BaseModel
class PlayerCreateData(BaseModel):
    player_name: str
    name: str = 'None'
    url: str = 'None'
    title: str = 'None'
    avatar: str = 'None'
    player_id: int = 0
    followers: int = 0
    country: str = 'None'
    location: str = 'None'
    joined: int = 0
    status: str = 'None'
    is_streamer: bool = False
    twitch_url: str = 'None'
    verified: bool = False
    league: str = 'None'

In [ ]:
a = PlayerCreateData(**{'player_name':'hikaru'})

In [ ]:
a

In [ ]:
a.model_dump()

In [ ]:
class Person(db.Model):
    id = db.Column(db.Integer, primary_key=True)

    user = db.relationship("BankSlip", foreign_keys='BankSlip.person_user_id', back_populates="person_user")
    reference = db.relationship("BankSlip", foreign_keys='BankSlip.person_ref_id', back_populates="person_reference")